In [1]:
import pandas as pd
import numpy as np
import gc
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import lightgbm
from bayes_opt import BayesianOptimization

In [2]:
data = pd.read_csv('cleandata/train.csv')
data.fillna(np.nan, inplace=True)
data.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in data.columns]
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [3]:
Y = data['TARGET']
X = data[list(data.columns[2:])]
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=0)
del data
gc.collect()

27

In [4]:
def bayes_optimization_gbm(X, Y, init_round=5, opt_round=15, n_folds=5, random_seed=0):
    train_data = lightgbm.Dataset(data=X, label=Y)
    # parameters
    def lgb_eval(n_estimators, learning_rate, num_leaves, min_data_in_leaf, max_depth, reg_alpha, reg_lambda, min_split_gain, min_child_weight,colsample_bytree,subsample):
        params = {'application':'binary', 'metric':'auc'}
        params['n_estimators'] = max(int(round(n_estimators)),500)
        params['learning_rate'] = min(learning_rate,0.3)
        params['num_leaves'] = int(round(num_leaves))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['max_depth'] = int(round(max_depth))
        params['reg_alpha'] = max(reg_alpha, 0)
        params['reg_lambda'] = max(reg_lambda, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight 
        params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
        params['subsample'] = max(min(subsample, 1), 0)
        
        cv=lightgbm.cv(params,train_data,nfold=n_folds,seed=random_seed,early_stopping_rounds=1000,stratified=True,verbose_eval=200,metrics=['auc'])
        return max(cv['auc-mean'])

    gbmbo = BayesianOptimization(lgb_eval, {'n_estimators': (2000, 10000),
                                            'learning_rate': (0.001, 0.1),
                                            'num_leaves': (25, 45),
                                            'min_data_in_leaf': (100, 1000),
                                            'max_depth': (5, 10),
                                            'reg_alpha': (0, 3),
                                            'reg_lambda': (0, 3),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50),
                                            'colsample_bytree': (0.1, 0.9),
                                            'subsample': (0.5, 1)},random_state=0)
    # optimize
    gbmbo.maximize(init_points=init_round, n_iter=opt_round)
    
    # return best parameters
    return gbmbo.max

best_params = bayes_optimization_gbm(x_train, y_train)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_da... | min_sp... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\alanl\Anaconda3\lib\site-packages\lightgbm\engine.py:503: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's auc: 0.786219 + 0.00343571
[400]	cv_agg's auc: 0.786828 + 0.00321705
[600]	cv_agg's auc: 0.78558 + 0.00332422
[800]	cv_agg's auc: 0.783813 + 0.00375874
[1000]	cv_agg's auc: 0.782268 + 0.00387711
[1200]	cv_agg's auc: 0.780584 + 0.00392295
|  1        |  0.787    |  0.5391   |  0.0718   |  8.014    |  29.52    |  481.3    |  0.06494  |  5.501e+0 |  42.84    |  2.891    |  1.15     |  0.8959   |


C:\Users\alanl\Anaconda3\lib\site-packages\lightgbm\engine.py:503: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's auc: 0.785941 + 0.00320549
[400]	cv_agg's auc: 0.787363 + 0.00310549
[600]	cv_agg's auc: 0.786947 + 0.00313445
[800]	cv_agg's auc: 0.786155 + 0.00321708
[1000]	cv_agg's auc: 0.785062 + 0.00319653
[1200]	cv_agg's auc: 0.783924 + 0.00323221
|  2        |  0.7875   |  0.5231   |  0.05724  |  9.628    |  8.197    |  178.4    |  0.003002 |  8.661e+0 |  40.56    |  2.61     |  2.936    |  0.8996   |


...

[200]	cv_agg's auc: 0.763655 + 0.00402959
[400]	cv_agg's auc: 0.777259 + 0.00411762
[600]	cv_agg's auc: 0.782758 + 0.00391994
[800]	cv_agg's auc: 0.785316 + 0.00369768
[1000]	cv_agg's auc: 0.786742 + 0.0035821
[1200]	cv_agg's auc: 0.787742 + 0.00349977
[1400]	cv_agg's auc: 0.788337 + 0.00348145
[1600]	cv_agg's auc: 0.788825 + 0.00349264
[1800]	cv_agg's auc: 0.789036 + 0.00343641
[2000]	cv_agg's auc: 0.789221 + 0.00340508
[2200]	cv_agg's auc: 0.789337 + 0.00344146
[2400]	cv_agg's auc: 0.789416 + 0.00352225
[2600]	cv_agg's auc: 0.789447 + 0.00361265
[2800]	cv_agg's auc: 0.789458 + 0.00360202
[3000]	cv_agg's auc: 0.789386 + 0.00358724
[3200]	cv_agg's auc: 0.789336 + 0.00353563
[3400]	cv_agg's auc: 0.789277 + 0.00352567
[3600]	cv_agg's auc: 0.789201 + 0.00351779
|  18       |  0.7895   |  0.6786   |  0.0123   |  8.85     |  47.33    |  993.3    |  0.08713  |  4.655e+0 |  28.04    |  0.822    |  1.436    |  0.6601   |


C:\Users\alanl\Anaconda3\lib\site-packages\lightgbm\engine.py:503: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's auc: 0.775337 + 0.0040584
[400]	cv_agg's auc: 0.784083 + 0.0038569
[600]	cv_agg's auc: 0.786822 + 0.00351738
[800]	cv_agg's auc: 0.788128 + 0.00354719
[1000]	cv_agg's auc: 0.788745 + 0.00358885
[1200]	cv_agg's auc: 0.789068 + 0.00364356
[1400]	cv_agg's auc: 0.789126 + 0.00364058
[1600]	cv_agg's auc: 0.789148 + 0.00370437
[1800]	cv_agg's auc: 0.78901 + 0.00364752
[2000]	cv_agg's auc: 0.788909 + 0.00374783
[2200]	cv_agg's auc: 0.788693 + 0.0038189
[2400]	cv_agg's auc: 0.788458 + 0.00396473
|  19       |  0.7892   |  0.6164   |  0.02309  |  7.995    |  49.85    |  980.9    |  0.06352  |  5.552e+0 |  25.87    |  2.335    |  2.635    |  0.5393   |


C:\Users\alanl\Anaconda3\lib\site-packages\lightgbm\engine.py:503: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	cv_agg's auc: 0.770235 + 0.00390553
[400]	cv_agg's auc: 0.779318 + 0.00401118
[600]	cv_agg's auc: 0.783306 + 0.00393738
[800]	cv_agg's auc: 0.785299 + 0.00382482
[1000]	cv_agg's auc: 0.786546 + 0.0037114
[1200]	cv_agg's auc: 0.787467 + 0.00369455
[1400]	cv_agg's auc: 0.788058 + 0.00366581
[1600]	cv_agg's auc: 0.788538 + 0.00354745
[1800]	cv_agg's auc: 0.788714 + 0.00354246
[2000]	cv_agg's auc: 0.788934 + 0.0035717
[2200]	cv_agg's auc: 0.789114 + 0.00359319
[2400]	cv_agg's auc: 0.789154 + 0.003617
[2600]	cv_agg's auc: 0.789162 + 0.00367435
[2800]	cv_agg's auc: 0.789129 + 0.00371337
[3000]	cv_agg's auc: 0.78909 + 0.00365137
[3200]	cv_agg's auc: 0.789088 + 0.00367409
[3400]	cv_agg's auc: 0.789003 + 0.00365174
[3600]	cv_agg's auc: 0.788918 + 0.00362479
|  20       |  0.7892   |  0.3499   |  0.01867  |  5.133    |  49.73    |  977.4    |  0.01299  |  7.175e+0 |  25.98    |  0.9443   |  1.841    |  0.5431   |
